In [1]:
print("=" * 60)
print("NOTEBOOK 3: TEST SCENARIO GENERATION")
print("=" * 60)

# Critical imports at the top
import torch
import json
from pathlib import Path
from datetime import datetime
import pandas as pd
from collections import Counter

print("\n✓ Core libraries imported")
print("=" * 60)

NOTEBOOK 3: TEST SCENARIO GENERATION

✓ Core libraries imported


In [2]:
print("\n" + "=" * 60)
print("LOADING PERSONAS AND PROTOTYPE DATA")
print("=" * 60)

# Load personas from Notebook 1
personas_file = Path("./personas_output/generated_personas_30.json")

if personas_file.exists():
    with open(personas_file, 'r') as f:
        personas = json.load(f)
    print(f"✓ Loaded {len(personas)} personas")
else:
    print("❌ Personas file not found!")
    print("Please run Notebook 1 first to generate personas")
    personas = []

# Load prototype from Notebook 2
prototype_file = Path("prototype_data.json")

if prototype_file.exists():
    with open(prototype_file, 'r') as f:
        prototype_data = json.load(f)
    print(f"✓ Loaded prototype with {prototype_data.get('screen_count', 0)} screens")
else:
    print("❌ Prototype file not found!")
    print("Please run Notebook 2 first to parse prototype")
    prototype_data = None

print("=" * 60)


LOADING PERSONAS AND PROTOTYPE DATA
✓ Loaded 30 personas
✓ Loaded prototype with 1 screens


In [3]:
print("\n" + "=" * 60)
print("SCENARIO GENERATION FUNCTIONS")
print("=" * 60)

def generate_test_scenarios_for_persona(persona, prototype, model=None, tokenizer=None):
    """
    Generate specific test scenarios for a persona
    
    Args:
        persona: Persona dictionary
        prototype: Prototype data
        model: Language model (optional)
        tokenizer: Tokenizer (optional)
    
    Returns:
        List of test scenarios
    """
    # Use global model if available
    if model is None:
        model = globals().get('model')
    if tokenizer is None:
        tokenizer = globals().get('tokenizer')
    
    scenarios = []
    
    # Extract persona characteristics
    user_type = persona.get('user_type', 'User')
    tech_level = persona.get('tech_proficiency', 'Intermediate')
    goal = persona.get('primary_goal', 'Complete task')
    pain_point = persona.get('pain_point', 'General usability')
    
    # Define scenario templates based on user characteristics
    scenario_templates = [
        {
            "task": f"Navigate to main feature and {goal.lower()}",
            "expected_behavior": f"{user_type} with {tech_level} tech skills successfully completes primary workflow",
            "success_criteria": ["Task completed within reasonable time", "No critical errors encountered"],
            "focus_area": "Primary workflow completion"
        },
        {
            "task": f"Recover from error state related to: {pain_point}",
            "expected_behavior": f"User encounters and resolves {pain_point} without abandoning task",
            "success_criteria": ["Error message is clear", "Recovery path is obvious"],
            "focus_area": "Error handling"
        },
        {
            "task": "Find and use help/information resources",
            "expected_behavior": f"{tech_level} user can locate assistance when needed",
            "success_criteria": ["Help is discoverable", "Information is understandable"],
            "focus_area": "Discoverability"
        },
        {
            "task": f"Complete secondary task: {goal}",
            "expected_behavior": "User navigates to and completes secondary feature",
            "success_criteria": ["Navigation is intuitive", "Task flow is logical"],
            "focus_area": "Secondary workflows"
        },
        {
            "task": "Use interface on first visit (no prior knowledge)",
            "expected_behavior": f"{user_type} understands interface without training",
            "success_criteria": ["Purpose is clear", "Next steps are obvious"],
            "focus_area": "Learnability"
        }
    ]
    
    # Create scenarios for this persona
    for i, template in enumerate(scenario_templates):
        scenario = {
            "scenario_id": f"{persona['id']}_scenario_{i+1}",
            "persona_id": persona['id'],
            "task_description": template["task"],
            "expected_behavior": template["expected_behavior"],
            "success_criteria": template["success_criteria"],
            "focus_area": template["focus_area"],
            "user_context": {
                "user_type": user_type,
                "tech_proficiency": tech_level,
                "primary_goal": goal,
                "pain_point": pain_point
            },
            "priority": "medium",
            "estimated_duration": "5-10 minutes"
        }
        
        scenarios.append(scenario)
    
    # Try to enhance with LLM if available
    if model and tokenizer:
        try:
            # Add LLM-generated context for first scenario
            prompt = f"""For a {user_type} with {tech_level} tech proficiency who wants to {goal}, 
what specific usability issues might they encounter with {pain_point}?
Provide 2-3 specific concerns in one sentence each."""

            inputs = tokenizer(
                prompt, 
                return_tensors="pt", 
                truncation=True, 
                max_length=256,
                padding=True
            )
            
            # Add attention mask
            inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])
            
            if torch.cuda.is_available():
                inputs = {k: v.to('cuda') for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.7,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=tokenizer.eos_token_id,
                    attention_mask=inputs['attention_mask']
                )
            
            enhanced_context = tokenizer.decode(outputs[0], skip_special_tokens=True)
            if prompt in enhanced_context:
                enhanced_context = enhanced_context.split(prompt)[-1].strip()
            
            scenarios[0]['llm_enhanced_context'] = enhanced_context[:200]
            
        except Exception as e:
            # Silently continue without LLM enhancement
            pass
    
    return scenarios


def prioritize_scenarios(scenarios):
    """
    Prioritize scenarios based on user characteristics and risk
    
    Args:
        scenarios: List of scenario dictionaries
    
    Returns:
        Scenarios with priority levels assigned
    """
    for scenario in scenarios:
        score = 0
        
        # Get user context
        context = scenario.get('user_context', {})
        tech_level = context.get('tech_proficiency', 'Intermediate').lower()
        focus = scenario.get('focus_area', '').lower()
        
        # High priority factors
        if tech_level in ['beginner', 'limited']:
            score += 3  # Beginners are high priority
        
        if focus in ['error handling', 'learnability']:
            score += 2  # Critical areas
        
        if 'accessibility' in str(scenario).lower():
            score += 2  # Accessibility is important
        
        if focus in ['primary workflow', 'main feature']:
            score += 2  # Core functionality
        
        # Medium priority factors
        if tech_level in ['intermediate', 'moderate']:
            score += 1
        
        if focus in ['secondary workflows', 'navigation']:
            score += 1
        
        # Assign priority based on score
        if score >= 5:
            scenario['priority'] = 'high'
        elif score >= 3:
            scenario['priority'] = 'medium'
        else:
            scenario['priority'] = 'low'
    
    return scenarios


def create_test_execution_plan(scenarios, personas):
    """
    Create an organized test execution plan
    
    Args:
        scenarios: List of scenarios
        personas: List of personas
    
    Returns:
        Test execution plan dictionary
    """
    # Group scenarios by priority
    high_priority = [s for s in scenarios if s['priority'] == 'high']
    medium_priority = [s for s in scenarios if s['priority'] == 'medium']
    low_priority = [s for s in scenarios if s['priority'] == 'low']
    
    # Calculate estimates
    total_scenarios = len(scenarios)
    avg_duration_minutes = 7.5  # Average of 5-10 minutes
    total_minutes = total_scenarios * avg_duration_minutes
    
    test_plan = {
        "plan_id": f"test_plan_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "created_at": datetime.now().isoformat(),
        "total_scenarios": total_scenarios,
        "total_personas": len(personas),
        "scenarios_by_priority": {
            "high": len(high_priority),
            "medium": len(medium_priority),
            "low": len(low_priority)
        },
        "estimated_duration": {
            "minutes": int(total_minutes),
            "hours": round(total_minutes / 60, 1)
        },
        "execution_order": {
            "phase_1_high_priority": high_priority,
            "phase_2_medium_priority": medium_priority,
            "phase_3_low_priority": low_priority
        },
        "all_scenarios": scenarios
    }
    
    return test_plan


print("✓ Scenario generation functions loaded")
print("=" * 60)


SCENARIO GENERATION FUNCTIONS
✓ Scenario generation functions loaded


In [4]:
print("\n" + "=" * 60)
print("GENERATING TEST SCENARIOS")
print("=" * 60)

if not personas:
    print("❌ No personas loaded. Cannot generate scenarios.")
    print("Please run Notebook 1 first.")
else:
    print(f"Generating scenarios for {len(personas)} personas...")
    print("This may take 2-3 minutes...\n")
    
    all_scenarios = []
    
    # Generate scenarios for each persona
    for i, persona in enumerate(personas):
        persona_scenarios = generate_test_scenarios_for_persona(
            persona, 
            prototype_data,
            model=globals().get('model'),
            tokenizer=globals().get('tokenizer')
        )
        
        all_scenarios.extend(persona_scenarios)
        
        # Progress indicator
        if (i + 1) % 5 == 0 or (i + 1) == len(personas):
            print(f"Progress: {i+1}/{len(personas)} personas processed ({len(all_scenarios)} scenarios generated)")
    
    print(f"\n✓ Generated {len(all_scenarios)} total test scenarios")
    print(f"✓ Average: {len(all_scenarios) / len(personas):.1f} scenarios per persona")
    print("=" * 60)


GENERATING TEST SCENARIOS
Generating scenarios for 30 personas...
This may take 2-3 minutes...

Progress: 5/30 personas processed (25 scenarios generated)
Progress: 10/30 personas processed (50 scenarios generated)
Progress: 15/30 personas processed (75 scenarios generated)
Progress: 20/30 personas processed (100 scenarios generated)
Progress: 25/30 personas processed (125 scenarios generated)
Progress: 30/30 personas processed (150 scenarios generated)

✓ Generated 150 total test scenarios
✓ Average: 5.0 scenarios per persona


In [5]:
print("\n" + "=" * 60)
print("PRIORITIZING TEST SCENARIOS")
print("=" * 60)

print("Analyzing and prioritizing scenarios...")

# Prioritize all scenarios
prioritized_scenarios = prioritize_scenarios(all_scenarios)

# Count by priority
priority_counts = {
    'high': sum(1 for s in prioritized_scenarios if s['priority'] == 'high'),
    'medium': sum(1 for s in prioritized_scenarios if s['priority'] == 'medium'),
    'low': sum(1 for s in prioritized_scenarios if s['priority'] == 'low')
}

print(f"\n✓ Scenarios prioritized")
print(f"\nPriority Distribution:")
print(f"  • High Priority: {priority_counts['high']} scenarios")
print(f"  • Medium Priority: {priority_counts['medium']} scenarios")
print(f"  • Low Priority: {priority_counts['low']} scenarios")

# Show top priority scenarios
high_priority_scenarios = [s for s in prioritized_scenarios if s['priority'] == 'high']

print("\n" + "-" * 60)
print("TOP 3 HIGH PRIORITY SCENARIOS:")
print("-" * 60)

for i, scenario in enumerate(high_priority_scenarios[:3]):
    print(f"\n{i+1}. {scenario['scenario_id']}")
    print(f"   Task: {scenario['task_description']}")
    print(f"   Focus: {scenario['focus_area']}")
    print(f"   User: {scenario['user_context']['user_type']} ({scenario['user_context']['tech_proficiency']})")

print("\n" + "=" * 60)


PRIORITIZING TEST SCENARIOS
Analyzing and prioritizing scenarios...

✓ Scenarios prioritized

Priority Distribution:
  • High Priority: 20 scenarios
  • Medium Priority: 47 scenarios
  • Low Priority: 83 scenarios

------------------------------------------------------------
TOP 3 HIGH PRIORITY SCENARIOS:
------------------------------------------------------------

1. persona_008_scenario_2
   Task: Recover from error state related to: Limited customization options
   Focus: Error handling
   User: Small Business Owner (Limited)

2. persona_008_scenario_5
   Task: Use interface on first visit (no prior knowledge)
   Focus: Learnability
   User: Small Business Owner (Limited)

3. persona_010_scenario_2
   Task: Recover from error state related to: Confusing terminology
   Focus: Error handling
   User: Creative Professional (Limited)



In [6]:
print("\n" + "=" * 60)
print("CREATING TEST EXECUTION PLAN")
print("=" * 60)

test_plan = create_test_execution_plan(prioritized_scenarios, personas)

print("✓ Test plan created")
print(f"\nTest Plan Summary:")
print(f"  • Total scenarios: {test_plan['total_scenarios']}")
print(f"  • Personas tested: {test_plan['total_personas']}")
print(f"  • High priority: {test_plan['scenarios_by_priority']['high']}")
print(f"  • Medium priority: {test_plan['scenarios_by_priority']['medium']}")
print(f"  • Low priority: {test_plan['scenarios_by_priority']['low']}")
print(f"  • Estimated time: {test_plan['estimated_duration']['minutes']} minutes")
print(f"    ({test_plan['estimated_duration']['hours']} hours)")

print("\n" + "=" * 60)


CREATING TEST EXECUTION PLAN
✓ Test plan created

Test Plan Summary:
  • Total scenarios: 150
  • Personas tested: 30
  • High priority: 20
  • Medium priority: 47
  • Low priority: 83
  • Estimated time: 1125 minutes
    (18.8 hours)



In [7]:
print("\n" + "="*60)
print("CREATING TEST EXECUTION PLAN")
print("="*60)

def create_test_plan(scenarios, max_scenarios=15):
    """
    Create structured test execution plan
    
    Args:
        scenarios: List of prioritized scenarios
        max_scenarios: Maximum scenarios to include in plan
    
    Returns:
        Dict with test plan structure
    """
    
    # Sort by priority
    priority_order = {'High': 0, 'Medium': 1, 'Low': 2, 'Unknown': 3}
    sorted_scenarios = sorted(
        scenarios,
        key=lambda x: priority_order.get(x.get('priority', 'Unknown'), 3)
    )[:max_scenarios]
    
    # Group by persona
    by_persona = {}
    for scenario in sorted_scenarios:
        persona_name = scenario.get('persona_name', 'Unknown')
        if persona_name not in by_persona:
            by_persona[persona_name] = []
        by_persona[persona_name].append(scenario)
    
    # Calculate estimated time
    total_time = sum([
        int(scenario.get('estimated_time', '10').split()[0])
        for scenario in sorted_scenarios
        if scenario.get('estimated_time')
    ])
    
    test_plan = {
        "plan_name": "Tandem Test Execution Plan",
        "created_date": str(pd.Timestamp.now()),
        "total_scenarios": len(sorted_scenarios),
        "estimated_time_minutes": total_time,
        "personas_tested": len(by_persona),
        "scenarios_by_persona": by_persona,
        "execution_order": [
            {
                "sequence": i,
                "scenario_id": scenario.get('task_description', ''),
                "persona": scenario.get('persona_name', ''),
                "priority": scenario.get('priority', ''),
                "estimated_minutes": scenario.get('estimated_time', 'Unknown')
            }
            for i, scenario in enumerate(sorted_scenarios, 1)
        ]
    }
    
    return test_plan

test_plan = create_test_plan(prioritized_scenarios, max_scenarios=15)

print(f"✓ Test plan created")
print(f"\nTest Plan Summary:")
print(f"  • Total scenarios: {test_plan['total_scenarios']}")
print(f"  • Personas tested: {test_plan['personas_tested']}")
print(f"  • Estimated time: {test_plan['estimated_time_minutes']} minutes")
print(f"  • ({test_plan['estimated_time_minutes'] / 60:.1f} hours)")

print("\n" + "="*60)


CREATING TEST EXECUTION PLAN
✓ Test plan created

Test Plan Summary:
  • Total scenarios: 15
  • Personas tested: 1
  • Estimated time: 0 minutes
  • (0.0 hours)



In [8]:
print("\n" + "="*60)
print("SAVING SCENARIOS AND TEST PLAN")
print("="*60)

# Save all scenarios
scenarios_path = "test_scenarios.json"
with open(scenarios_path, 'w') as f:
    json.dump(prioritized_scenarios, f, indent=2)
print(f"✓ Saved {len(prioritized_scenarios)} scenarios to {scenarios_path}")

# Save test plan
plan_path = "test_execution_plan.json"
with open(plan_path, 'w') as f:
    json.dump(test_plan, f, indent=2)
print(f"✓ Saved test plan to {plan_path}")

# Create human-readable summary
summary_path = "test_plan_summary.txt"
with open(summary_path, 'w') as f:
    f.write("TANDEM TEST EXECUTION PLAN\n")
    f.write("="*60 + "\n\n")
    f.write(f"Created: {test_plan['created_date']}\n")
    f.write(f"Total Scenarios: {test_plan['total_scenarios']}\n")
    f.write(f"Personas: {test_plan['personas_tested']}\n")
    f.write(f"Estimated Time: {test_plan['estimated_time_minutes']} minutes\n\n")
    
    f.write("EXECUTION ORDER:\n")
    f.write("-"*60 + "\n\n")
    
    for item in test_plan['execution_order']:
        f.write(f"{item['sequence']}. {item['scenario_id']}\n")
        f.write(f"   Persona: {item['persona']} | Priority: {item['priority']} | Time: {item['estimated_minutes']}\n\n")

print(f"✓ Saved readable summary to {summary_path}")

print(f"\n✓ Notebook 3 Complete!")
print(f"\nGenerated:")
print(f"  • {len(prioritized_scenarios)} test scenarios")
print(f"  • Test plan for {test_plan['total_scenarios']} priority scenarios")
print(f"  • Covers {test_plan['personas_tested']} different personas")
print(f"\nNext: Open Notebook 4 to run AI-powered usability testing")
print("="*60)


SAVING SCENARIOS AND TEST PLAN
✓ Saved 150 scenarios to test_scenarios.json
✓ Saved test plan to test_execution_plan.json
✓ Saved readable summary to test_plan_summary.txt

✓ Notebook 3 Complete!

Generated:
  • 150 test scenarios
  • Test plan for 15 priority scenarios
  • Covers 1 different personas

Next: Open Notebook 4 to run AI-powered usability testing
